# Image classification on [MNIST](https://yann.lecun.com/exdb/mnist/) dataset

_Handling Convolutional Neural Networks_

---


In this tutorial you will learn to : 
* Write multilayer perceptron and convolutional network with [`Keras`](https://keras.io/) and [`Tensorflow`](https://www.tensorflow.org/)
* Understand how `convolutional`, `max pooling`, `stride` and `padding` layers work.
* Use these models for image classification.

## Libraries

In [ ]:
import tensorflow.keras.utils as ku
import tensorflow.keras.models as km
import tensorflow.keras.layers as kl
import tensorflow.keras.optimizers as ko
import tensorflow.keras.preprocessing.image as k

In [ ]:
import pandas as pd
import numpy as np
import time

%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set()

In [ ]:
import tensorflow as tf
tf.__version__

This code lines allow you to check if your computer is using CPU or GPU ressources. <br>
**Warning** : You won't be able to use GPU if another notebook is open and still uses GPU.

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## Data exploration

The dataset that will be used in this TP is the [MNIST DataBase](http://yann.lecun.com/exdb/mnist/).<br>
It is composed of 70.000 images (60.000 for learning, 10.000 for test) of 28x28 pixels of handwritten digits from 0 to 9.<br>

These data are directly available on the `Keras` library.

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

##### <span style="color:purple">**Todo:** Check that the images are the correct size, as well as the test and train sets.</span>

In [ ]:
### TO BE COMPLETED ###

N_x_pixels = ...
N_y_pixels = ...
N_train = ...
N_test = ...
N_classes = ...

print("Train data: %d images  (%d x %d pixels)" %(N_train, N_x_pixels, N_y_pixels))
print("Test data: %d images  (%d x %d pixels)" %(N_test, N_x_pixels, N_y_pixels))
print("Number of classes: %d classes" %N_classes)

In [ ]:
# %load solutions/MNIST/size.py

##### <span style="color:purple">**Question:**  Is the dataset balanced?</span>

In [ ]:
### TO BE COMPLETED ###

[...]

In [ ]:
# %load solutions/MNIST/histograms.py

### Data visualization

In [ ]:
import random as rd

##### <span style="color:purple">**Todo:** View an example of each digit</span>

In [ ]:
### TO BE COMPLETED ###

[...]

In [ ]:
# %load solutions/MNIST/imshow.py

## Image classification with Multi Layer Perceptron model.

We will first try to learn an image classifier with a MLP model with the following architecture.

* A Dense layer with 128 neurons and *relu* activation function
* A Dropout Layer with 20% drop rate
* A Dense layer with 128 neurons and *relu* activation function
* A Dropout Layer with 20% drop rate
* A Dense layer with 10 neurons (Number of classes ) and *softmax* activation function

### Data format

Some modifications are required on the data to use them with our model. 

The first layer is a Dense Layer, which handles 1D vectors as an input. We must first reshape the 2D $28\times28$ images as a 1D $28\times28=784$ vector. 
<!-- We take this opportunity to renormalize the image, _i.e._ divide all its values by $255$ (grayscale image). -->

In [ ]:
x_train_flatten = x_train.reshape((N_train, N_x_pixels*N_y_pixels))
x_test_flatten = x_test.reshape((N_test, N_x_pixels*N_y_pixels))
N_dim_flatten = x_train_flatten.shape[1]

print("Dimensions of flatten train images: %d x %d" %(x_train_flatten.shape))
print("Dimensions of flatten test images: %d x %d" %(x_test_flatten.shape))

### Architecture 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout

##### <span style="color:purple">**Todo:** Complete the code below to define the previously described network.</span>

The `Keras` $\texttt{Sequential}$ method builds neural networks by juxtaposing layers. 

In [ ]:
### TO BE COMPLETED ###

# Network definition
mlp = Sequential()
mlp.add( Input(shape=(N_dim_flatten,)) )
mlp.add( Dense(128, activation='relu') )
mlp.add( Dropout(0.2) )

[...]

# Summary
mlp.summary()

In [ ]:
# %load solutions/MNIST/mlp.py

The above summary displays the number of pararameters/weigths of the model.

##### <span style="color:purple">**Todo:** Retrieve these values with the formulas seen in the course.</span>

### Training

In [ ]:
from tensorflow.keras.optimizers import RMSprop

You will now instantiate your model by defining :
* An _optimizer_: $\texttt{RMSprop}$
* A _loss_ function: $\texttt{Categorical crossentropy}$
* A _metric_: This argument is an option, it allows to compute the metric if you want to check the evolution of the training. Here we choose to compute the accuracy during the training.
<br><br>

> **Remark**: In Keras you can choose either $\texttt{sparse\_categorical\_crossentropy}$ or $\texttt{categorical\_crossentropy}$ loss.
> * The former handles 1D ($N\times1$) vectors where each entry contains the label of the data, _i.e_ $[0,3,5,9,3,4,\ldots]$.
> * The latter handles only one-hot encoding of this vector, ie  2D vectors ($N\times N_{classes}$) matrices.
>  
> Keras has a $\texttt{to\_categorical}$ function which allows to convert a vector to its one-hot encoding representation.

In [ ]:
mlp.compile(loss = 'sparse_categorical_crossentropy',
            optimizer = RMSprop(),
            metrics = ['accuracy'])

> **Remark**: if you want to restart network training, _you need to reset its weights._

To do this, you need to re-execute the previous cells from the network definition! 
Otherwise, you risk restarting your optimization procedure from a previous run (which may or may not have gone well) and misinterpreting your new results.

In [ ]:
batch_size = 256
epochs = 10

t_train_mlp = time.time()
history = mlp.fit(x_train_flatten, y_train,
                  batch_size = batch_size,
                  epochs = epochs,
                  verbose = 1,
                  validation_data = (x_test_flatten, y_test))
t_train_mlp = t_train_mlp - time.time()

### Results

In [ ]:
from sklearn.metrics import confusion_matrix

##### <span style="color:purple">**Todo:** Evaluate the performance of this training.</span>

You can visualize a confusion matrix between the predictions for $y$ and the true values.

In [ ]:
### TO BE COMPLETED ###

score_mlp = mlp.evaluate(x_test_flatten, y_test, verbose=0)
predict_mlp = mlp.predict(x_test_flatten)

print('Test loss:', ...)
print('Test accuracy:', ...)
print("Running time: %.2f seconds" %t_train_mlp)

[...]

In [ ]:
# %load solutions/MNIST/mlp_results.py

##### <span style="color:purple">**Question:** What can you say about these results?</span>

##### <span style="color:purple">**Exercise:** Normalize the data in order to have values between 0 and 1 and run again the learning.</span>

What can you say about these results?

In [ ]:
### TO BE COMPLETED ###

[...]

In [ ]:
# %load solutions/MNIST/mlp_norm.py

## Convolutional Layers

In this part we will use convolution layers to build a convolutional classifier.

### Data format

The convolution architecture takes as input images and not 1D vectors. However, some data formating are still required.

A third dimension is required : the $\texttt{channels}$ dimension which will allow to describe each pixel. In our case this dimension's size is only 1 because the images are only defined with grey scale. However for colour images, each pixel is coded with several values (Images are generally encoded with 3 values (RGB channels)). 

Hence, we need to reshape the images from a $28\times28$ dimension to a $28\times28\times1$ dimension

In [ ]:
x_train_conv = np.expand_dims(x_train, axis=-1)
x_test_conv = np.expand_dims(x_test, axis=-1)

print("Train data: %d images" %x_train_conv.shape[0], x_train_conv.shape[1:])
print("Test data: %d images" %x_test_conv.shape[0], x_test_conv.shape[1:])

### Edge detection

We will first check the transformation applied by a convolution layer.

In [ ]:
from tensorflow.keras.layers import Conv2D

We select an example for each of the digits ($0$ to $9$), and define a new test image (a $+$) so that we can observe the effect of the convolution filters.
Try testing the proposed filters (or even others!) on different images, digit or plus.

In [ ]:
fig = plt.figure(figsize=(11, 5))

# Plus image test
img_plus = np.zeros((28,28), dtype=int)
img_plus[4:24,11:17] = 1
img_plus[11:17,4:24] = 1
img_plus = np.expand_dims(img_plus, axis=-1)

ax = fig.add_subplot(1, 11, 11)
ax.imshow(img_plus[:,:,0], cmap=plt.cm.gray_r)
ax.grid(False)
ax.axis('off')


# Digits
sample_index = np.zeros(10, dtype=int)
for i in range(10):
    sample_index[i] = np.where(y_train==i)[0][0]
    ax = fig.add_subplot(1, 11, i+1)
    ax.imshow(x_train[sample_index[i]], cmap=plt.cm.gray_r, interpolation='nearest')
    ax.grid(False)
    ax.axis('off')


plt.tight_layout()
plt.show()

Here are some examples of $3\times3$ convolution filters.

In [ ]:
conv_filter_1 = np.array([
        [0.2, -0.2, 0],
        [0.2, -0.2, 0],
        [0.2, -0.2, 0],
    ])

conv_filter_2 = 1/9 * np.array([
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
    ])

conv_filter_3 = np.array([
        [0, -1, 0],
        [-1, 5, -1],
        [0, -1, 0],
    ])

conv_filter_4 = np.array([
        [-1, 2, -1],
        [-1, 2, -1],
        [-1, 2, -1],
    ])

conv_filter_5 = np.array([
        [-1, -1, 2],
        [-1, 2, -1],
        [2, -1, -1],
    ])

In the following code, we define a convolutional network with  only one filter for which we manually define the weights.

In [ ]:
conv_filter = conv_filter_1

def my_init_filter(shape, conv_filter=conv_filter, dtype=None, partition_info=None):
    xf,yf = conv_filter.shape
    array = conv_filter.reshape(xf, yf, 1, 1)
    return array
print(my_init_filter(0, conv_filter_2).shape)

conv_layer = Sequential()
conv_layer.add( Input(shape=(28, 28, 1)) )
conv_layer.add( Conv2D(kernel_size=(3,3), filters=1, kernel_initializer=my_init_filter) )

Note that in  $\texttt{my\_init\_filter}$ two dimensions have been added to the conv filter.

##### <span style="color:purple">**Question:** What do these dimensions represent?</span>

 The following codes allow to display the image, the filter and the convoluted image.

In [ ]:
def build_conv_layer(conv_filter):    
    def my_init_filter(shape, conv_filter=conv_filter, dtype=None, partition_info=None):
        xf,yf = conv_filter.shape
        array = conv_filter.reshape(xf, yf, 1, 1)
        return array
    
    conv_layer = Sequential()
    conv_layer.add( Input(shape=(28, 28, 1)) )
    conv_layer.add( Conv2D(kernel_size=(3,3), filters=1, kernel_initializer=my_init_filter) )
    return conv_layer

In [ ]:
# CHOICES

# Image choice : Digit or Plus
idx = sample_index[9] #####
x = x_train_conv[idx] #####
# x = img_plus #####

# Filter choice
conv_filter = conv_filter_3 #####
conv_layer = build_conv_layer(conv_filter)

# --- #

img_in = np.expand_dims(x, 0)
img_out = conv_layer.predict(img_in)

# Original image
fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(15, 5))
ax0.imshow(img_in[0,:,:,0], cmap=plt.cm.gray_r)
ax0.set_title("Original image")
ax0.grid(False)

# Filter
norm_conv_filter = (conv_filter-conv_filter.min())/conv_filter.max()
ax1.imshow(norm_conv_filter.astype(np.uint8), cmap=plt.cm.gray_r)   # "binary"
ax1.set_title("Filter")
ax1.grid(False)

# Filtered image
ax2.imshow(img_out[0,:,:,0].astype(np.uint8), cmap=plt.cm.gray_r)
ax2.set_title("Filtered image")
ax2.grid(False)

##### <span style="color:purple">**Question:** What do you see?</span>

* Are the output image coherent according to the designed filter ?
* How do the proposed filters affect the image?
* Change the code in order to test different filters (to detect horizontal edges, etc...)

### Strides and Padding

We will now study the effect on $\texttt{strides}$ and $\texttt{padding}$ arguments on the image.

$\texttt{padding}$ can take the values $\texttt{"same"}$ or $\texttt{"valid"}$. $\texttt{"valid"}$ means no padding.

In [ ]:
def build_conv_layer_sp(conv_filter, strides=2, padding="same"):    
    def my_init_filter(shape, conv_filter=conv_filter, dtype=None, partition_info=None):
        xf,yf = conv_filter.shape
        array = conv_filter.reshape(xf, yf, 1, 1)
        return array
    
    conv_layer = Sequential()
    conv_layer.add( Input(shape=(28, 28, 1)) )
    conv_layer.add( Conv2D(kernel_size=(5,5), filters=1, kernel_initializer=my_init_filter,
                  strides=strides, padding=padding) )  ### NEW
    return conv_layer

In [ ]:
# CHOICES

# Image choice : Digit or Plus
idx = sample_index[9] #####
x = x_train_conv[idx] #####
# x = img_plus #####

# Filter choice
conv_filter = conv_filter_3 #####
conv_layer = build_conv_layer_sp(conv_filter, padding="valid")

# --- #

img_in = np.expand_dims(x, 0)
img_out = conv_layer.predict(img_in)

fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(15, 5))
ax0.imshow(img_in[0,:,:,0].astype(np.uint8), cmap=plt.cm.gray_r);
ax0.set_title("Original image")
ax0.grid(False)

norm_conv_filter = (conv_filter-conv_filter.min())/conv_filter.max()
ax1.imshow(norm_conv_filter.astype(np.uint8), cmap=plt.cm.gray_r);
ax1.set_title("Filter")
ax1.grid(False)

ax2.imshow(img_out[0,:,:,0].astype(np.uint8), cmap=plt.cm.gray_r);
ax2.set_title("Filtered image")
ax2.grid(False)

##### <span style="color:purple">**Question:** What do you see?</span>

* Check the dimension of the output images. Are they coherent? <br>
* Change both *strides* and *padding* arguments and understand the effect of these changes.

### Max Pooling

In [ ]:
from tensorflow.keras.layers import MaxPool2D, MaxPooling2D

##### <span style="color:purple">**Exercise:** Write a similar code than above to check and understand the behaviour of the $\texttt{max pooling}$ layer.</span>

In [ ]:
### TO BE COMPLETED ###

[...]

In [ ]:
# %load solutions/MNIST/max_pooling.py

##### <span style="color:purple">**Question:** What are the dimension of the output image?</span>

## Convolutional Neural Network (CNN or ConvNet)

We will now build convolutional networks and see the performances on this kind of model on  image classification problems.

### LeNet5

We first test the  [LeNet5](https://en.wikipedia.org/wiki/LeNet) model, proposed by _LeCun et al._

In [ ]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adadelta

In [ ]:
LeNet = Sequential()
LeNet.add(Input(shape=(28,28,1)))

LeNet.add(Conv2D(filters = 6, kernel_size = 5, strides = 1, activation = 'tanh'))
LeNet.add(MaxPool2D(pool_size = 2, strides = 2))

LeNet.add(Conv2D(filters = 16, kernel_size = 5,strides = 1, activation = 'tanh'))
LeNet.add(MaxPool2D(pool_size = 2, strides = 2))

LeNet.add(Flatten())
LeNet.add(Dense(units = 120, activation = 'tanh'))
LeNet.add(Dense(units = 84, activation = 'tanh'))
LeNet.add(Dense(units = 10, activation = 'softmax'))

LeNet.summary()

##### <span style="color:purple">**Exercise:** Retrieve 'manually' the number of parameters of this model.</span>

What can you say about the total number of parameters compared with the MLP model defined before? Which layer has the highest number of parameters?

#### Training

In [ ]:
batch_size=128
epochs=10

LeNet.compile(loss = "sparse_categorical_crossentropy",
              optimizer = Adadelta(),
              metrics = ['accuracy'])

t_train_LeNet = time.time()
LeNet.fit(x_train_conv, y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose = 1,
          validation_data = (x_test_conv, y_test))
t_train_LeNet = time.time() - t_train_LeNet

##### <span style="color:purple">**Question:** Why is the training time longer?</span>

##### <span style="color:purple">**Exercise:** Compare the accuracy with the one obtained with the optimizer $\texttt{Adam}$. </span>

#### Results

In [ ]:
score_LeNet = LeNet.evaluate(x_test_conv, y_test, verbose=0)
predict_LeNet = LeNet.predict(x_test_conv)

print('Test loss:', score_LeNet[0])
print('Test accuracy:', score_LeNet[1])
print("Time Running: %.2f seconds" %t_train_LeNet )

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(1,1,1)
ax = sns.heatmap(pd.DataFrame(confusion_matrix(y_test, predict_LeNet.argmax(1))), annot=True, fmt="d")

### A more complex architecture


We will now design a more complex architecture to try to improve the results of the classification :

* A $\texttt{Conv2D}$ layer with $32 - 3\times3$ filters and the $\texttt{Relu}$ activation function,
* A $\texttt{Conv2D}$ layer with $64 - 3\times3$ filters and the $\texttt{Relu}$ activation function,
* A $\texttt{MaxPooling}$ layer with a $2\times2$ window,
* A $\texttt{Dropout}$ layer with a $25%$ drop rate,
* A $\texttt{Flatten}$ layer,
* A $\texttt{Dense}$ layer with $128$ neurons  and the $\texttt{Relu}$ activation function,
* A $\texttt{Dropout}$ layer with a $50\%$ drop rate,
* A $\texttt{Dense}$ layer with $10$ neurons  and the $\texttt{softmax}$ activation function.


##### <span style="color:purple">**Exercise:** Define this model and train it.</span>

In [ ]:
### TO BE COMPLETED ###

[...]

In [ ]:
# %load solutions/MNIST/cnn.py

##### <span style="color:purple">**Exercise:** Compare the accuracy with the one obtained with the optimizer Adam.</span>

In [ ]:
### TO BE COMPLETED ###

[...]

In [ ]:
# %load solutions/MNIST/cnn_results.py

##### <span style="color:purple">**Exercise:** Comment the results.</span>